In [1]:
import os
import sys

# Build an absolute path from this notebook's parent directory
module_path = os.path.abspath(os.path.join('..'))

# Add to sys.path if not already present
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from src.utils import hello_world

In [3]:
hello_world.say_hello()

Hello Jane
